<a href="https://colab.research.google.com/github/Dhyanesh-Panchal/GSR-research-and-stress-detection/blob/master/notebooks/DNN_GSR_phasic_window_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras


---
### Prepare the the Data

In [ ]:
data_df = pd.read_csv("https://raw.githubusercontent.com/Dhyanesh-Panchal/GSR-research-and-stress-detection/master/preprocessed_data/sub_1.csv").drop("Unnamed: 0",axis=1)
data_df.head()

,time,gsr,video,valence,arousal,EDA_Clean,EDA_Phasic,EDA_Tonic,class_4,class_2_arousal
0,0,11.749,10,5.0,5.0,11.749013,-0.060830,11.809843,LL,L
1,50,11.729,10,5.0,5.0,11.738831,-0.071808,11.810639,LL,L
2,100,11.737,10,5.0,5.0,11.729231,-0.082219,11.811450,LL,L
3,150,11.713,10,5.0,5.0,11.720394,-0.091883,11.812276,LL,L
4,200,11.717,10,5.0,5.0,11.712337,-0.100781,11.813118,LL,L


In [ ]:
for i in range(2,31):
    df = pd.read_csv(f"https://raw.githubusercontent.com/Dhyanesh-Panchal/GSR-research-and-stress-detection/master/preprocessed_data/sub_{i}.csv").drop("Unnamed: 0",axis=1)
    data_df = pd.concat([data_df,df],axis=0)

In [ ]:
data_df

,time,gsr,video,valence,arousal,EDA_Clean,EDA_Phasic,EDA_Tonic,class_4,class_2_arousal
0,0,11.749,10,5.0,5.0,11.749013,-0.060830,11.809843,LL,L
1,50,11.729,10,5.0,5.0,11.738831,-0.071808,11.810639,LL,L
2,100,11.737,10,5.0,5.0,11.729231,-0.082219,11.811450,LL,L
3,150,11.713,10,5.0,5.0,11.720394,-0.091883,11.812276,LL,L
4,200,11.717,10,5.0,5.0,11.712337,-0.100781,11.813118,LL,L
...,...,...,...,...,...,...,...,...,...,...
49027,2451350,7.243,12,5.0,5.0,7.236746,-0.036511,7.262909,LL,L
49028,2451400,7.223,12,5.0,5.0,7.243913,-0.030023,7.262900,LL,L
49029,2451450,7.258,12,5.0,5.0,7.250596,-0.024032,7.262893,LL,L
49030,2451500,7.259,12,5.0,5.0,7.256530,-0.018805,7.262888,LL,L


In [ ]:
filtered_data = data_df[(data_df["arousal"]!=5) & (data_df["valence"]!=5)]
filtered_data = filtered_data.reset_index().drop("index",axis=1)
filtered_data

,time,gsr,video,valence,arousal,EDA_Clean,EDA_Phasic,EDA_Tonic,class_4,class_2_arousal
0,10700,11.978,10,4.972,4.721,11.967936,-0.000717,11.968653,LL,L
1,10750,11.974,10,4.951,4.708,11.967650,-0.000071,11.967722,LL,L
2,10800,11.958,10,4.951,4.704,11.966279,-0.000498,11.966777,LL,L
3,10850,11.958,10,4.953,4.699,11.964547,-0.001272,11.965818,LL,L
4,10900,11.974,10,4.955,4.690,11.963083,-0.001763,11.964846,LL,L
...,...,...,...,...,...,...,...,...,...,...
25819,2333850,7.002,12,4.863,5.359,7.008476,0.003220,7.005256,LH,H
25820,2333900,6.994,12,4.863,5.327,7.007426,0.000460,7.006966,LH,H
25821,2333950,7.026,12,4.863,5.314,7.010802,0.002130,7.008672,LH,H
25822,2334000,6.999,12,4.863,5.292,7.017426,0.007053,7.010373,LH,H


In [ ]:
# windowing function
def rolling_window(df,window_size):
    windows = []
    for i in range(len(df)-window_size+1):
        single_window = df.iloc[i:i+window_size]
        # print(single_window)
        windows.append(single_window)
    return windows

In [ ]:
def pre_process(df):
  windowed_data = rolling_window(df,100)
  intermediate_array = []
  for frame in windowed_data:
    intermediate_array.append(list(frame["EDA_Phasic"]))

  phasic_windowed = pd.DataFrame(np.array(intermediate_array))
  phasic_windowed.shape

  X = np.array(phasic_windowed).reshape(len(phasic_windowed),100,1)
  # inserted the initial arousal value during the "start" of window
  # X = np.concatenate((X,np.array(df['class_2_arousal'][99:]).reshape(len(df['class_2_arousal']) - 99,1,1)) , axis=1)

  y = np.array(df["class_2_arousal"])[99:]

  return X,y

In [ ]:
X,y = pre_process(filtered_data)

In [ ]:
X.shape

(25725, 100, 1)

In [ ]:
y.shape

(25725,)

In [ ]:
y = pd.get_dummies(y)

In [ ]:
import tensorflow as tf

model = tf.keras.models.Sequential([
  tf.keras.layers.Conv1D(64, 10, activation='relu', input_shape=(100, 1)),
  tf.keras.layers.MaxPooling1D(2),
  tf.keras.layers.Conv1D(128, 7, activation='relu'),  # Additional Conv1D layer
  tf.keras.layers.MaxPooling1D(2),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(64, activation='relu'),
  tf.keras.layers.Dense(2, activation='softmax')  # Adjusted for 2-class classification
])

# Compile the model for a classification task
model.compile(loss='binary_crossentropy',  # Changed to categorical_crossentropy
              optimizer='adam',
              metrics=['accuracy'])  # Common to include accuracy as a metric for classification


In [ ]:
fitting = model.fit(X, y, epochs=15)  #Just test

Epoch 1/15
804/804 [==============================] - 4s 5ms/step - loss: 0.4216 - accuracy: 0.7835
Epoch 2/15
804/804 [==============================] - 3s 4ms/step - loss: 0.3653 - accuracy: 0.8160
Epoch 3/15
804/804 [==============================] - 3s 4ms/step - loss: 0.3027 - accuracy: 0.8546
Epoch 4/15
804/804 [==============================] - 4s 5ms/step - loss: 0.2293 - accuracy: 0.8972
Epoch 5/15
804/804 [==============================] - 3s 4ms/step - loss: 0.1739 - accuracy: 0.9258
Epoch 6/15
804/804 [==============================] - 3s 4ms/step - loss: 0.1206 - accuracy: 0.9515
Epoch 7/15
804/804 [==============================] - 4s 4ms/step - loss: 0.0838 - accuracy: 0.9702
Epoch 8/15
804/804 [==============================] - 4s 5ms/step - loss: 0.0617 - accuracy: 0.9778
Epoch 9/15
804/804 [==============================] - 3s 4ms/step - loss: 0.0420 - accuracy: 0.9876
Epoch 10/15
804/804 [==============================] - 3s 4ms/step - loss: 0.0394 - accuracy: 0.9879